## Bioinformatics: Sequence Alignment
Sequence alignment in bioinformatics is the process of comparing DNA, RNA, or protein sequences to find similarities. It helps scientists understand relationships, functions, or evolutionary history. By lining up sequences, we can spot matches, differences, and important regions.

In [ ]:
from collections import Counter
import random
import numpy as np

## Bioinformatics II: Sequence Alignment 

References:
Jones and Pevzner 2004, An Introduction to Bioinformatics Algorithms

# Edit distance
Finds how dissimilar two strings are to one another by calculating the minimum number of edits needed to make the two identical. 

# Similarity Searches in bioinformatics
- DNA sequence comparison: when biologists infer a newly sequenced gene’s function by finding similarities with genes of known function. 
- DNA mutation is caused by DNA replication errors leading to substitutions, insertions, and deletions of nucleotides. 

- In 1984, scientists found v-sis oncogene, a cancer-causing gene. They compared it with all known genes from that time, and it matched with a normal growth and development gene. They hypothesized that cancer was caused by the normal growth gene doing its job at the wrong time. 
- Similarly, the Cystic fibrosis gene was discovered through a successful similarity search as well. Cystic fibrosis is a fatal disease which damages body organs and causes thick, lung clogging mucus. 

## Consensus string (dunno if this should be included)
Gets the most common nucleobases per index and then use it in the final string

In [ ]:
def consensus_string(matrix):
    consensus = []
    num_columns = len(matrix[0])

    for col in range(num_columns):
        column_bases = [row[col] for row in matrix]   
        counts = Counter(column_bases)                
        most_common = counts.most_common(1)[0][0]     
        consensus.append(most_common)

    return ''.join(consensus)

In [ ]:
P = generate_matrix()

for row in P:
    print(" ".join(row))

consensus_string(P)

## 1. Hamming Distance
The number of positions that differ in two strings of equal lengths. 
The symbols may be letters, bits, or decimal digits, among other possibilities. 
For example, the Hamming distance between:
- "karolin" and "kathrin" is 3.
- "karolin" and "kerstin" is 3.
- "kathrin" and "kerstin" is 4.
- 0000 and 1111 is 4.
- 2173896 and 2233796 is 3.

In [ ]:
a = ['A', 'T', 'T', 'G', 'T', 'C']
b = ['A', 'C', 'T', 'C', 'T', 'C']

distance = 0

print(f'Distance: {distance}')

for i in range (len(a)):
  if a[i] != b[i]:
    distance += 1

print(f'Hamming distance: {distance}')

In [ ]:
# Counts how many nucleobases (A,T,C,G) are different from the REFERENCE SEQUENCE in the first row 

#generate dna sequences of the same length
def generate_dna_sequences_samelength():
    bases = ['A', 'T', 'C', 'G']
    matrix_rows = random.randint(5, 10) #number of dna sequences

    sequences = []
    length = random.randint(5,8)
    for row in range(matrix_rows):
        sequence = [random.choice(bases) for column in range(length)]
        sequences.append(sequence)

    return sequences

X = generate_dna_sequences_samelength()

for row in X:
    print(" ".join(row))

In [ ]:
def Hamming_distance(matrix, reference_index=0):
    reference = matrix[reference_index] #compare to the first row
    distance = 0
    for row in matrix:
        for i in range(len(reference)):
            if row[i] != reference[i]:
                distance += 1
    return distance

Hamming_distance(X)

However, it only works with sequences of identical lengths

In [ ]:
def generate_random_dna_sequences():
    bases = ['A', 'T', 'C', 'G']
    num_sequences = random.randint(5, 10)

    sequences = []
    for _ in range(num_sequences):
        length = random.randint(5, 15)
        sequence = [random.choice(bases) for _ in range(length)]
        sequences.append(sequence)

    return sequences

random_dna = generate_random_dna_sequences()
for i, seq in enumerate(random_dna, 1):
    print(seq)

X = generate_random_dna_sequences()
Hamming_distance(X)


## 2. Levenshtein 
The levevnshtein edit distance works for strings with different lengths

In [ ]:
def levenshtein(string_one, string_two):
    m = len(string_one)
    n = len(string_two)

    '''
    if m == 0:
        return n
    if n == 0:
        return m
    '''

    dp = [[0]*(n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if string_one[i - 1] == string_two[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min([
                                    dp[i][j - 1], 
                                    dp[i - 1][j], 
                                    dp[i - 1][j - 1]
                                    ])
    return dp[m][n]

In [ ]:
L = generate_random_dna_sequences()

for i, seq in enumerate(L, 1):
    print(seq)

n = len(L)
dist_matrix = np.zeros((n, n), dtype=int)

for i in range(n):
    for j in range(n):
        dist_matrix[i, j] = levenshtein(L[i], L[j])

# Display the distance matrix
print("\nLevenshtein Distance Matrix:")
for row in dist_matrix:
    print(' '.join(map(str, row)))


## 3. Multisequence
Description

## 4. Local

### 4.1 Concept
- Finds the best scoring **local** alignment (subsequences)
- Initialization starts with zeros
- Traceback starts at the **highest scoring cell**

### 4.2 Code

In [ ]:
def smith_waterman(seq1, seq2, match=3, mismatch=-3, gap=-2):
    n, m = len(seq1), len(seq2)
    score = np.zeros((n+1, m+1), dtype=int)
    max_score = 0
    max_pos = (0, 0)

    # Fill scoring matrix
    for i in range(1, n+1):
        for j in range(1, m+1):
            diag = score[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            delete = score[i-1][j] + gap
            insert = score[i][j-1] + gap
            score[i][j] = max(0, diag, delete, insert)
            if score[i][j] >= max_score:
                max_score = score[i][j]
                max_pos = (i, j)

    # Traceback
    aligned_seq1, aligned_seq2 = '', ''
    i, j = max_pos
    while score[i][j] > 0:
        current = score[i][j]
        if current == score[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch):
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif current == score[i-1][j] + gap:
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1

    return score, aligned_seq1, aligned_seq2

# Example
score_matrix, a1, a2 = smith_waterman("ACACACTA", "AGCACACA")
print("\nLocal Alignment:")
print(a1)
print(a2)

In [ ]:
def smith_waterman(seq1, seq2, match=3, mismatch=-3, gap=-2):
    n, m = len(seq1), len(seq2)
    score = np.zeros((n + 1, m + 1), dtype=int)
    max_score = 0
    max_pos = (0, 0)

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            diag = score[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            delete = score[i - 1][j] + gap
            insert = score[i][j - 1] + gap
            score[i][j] = max(0, diag, delete, insert)

            if score[i][j] > max_score:
                max_score = score[i][j]
                max_pos = (i, j)

    return score, max_pos

def plot_score_matrix(matrix, seq1, seq2, max_pos):
    fig, ax = plt.subplots(figsize=(10, 6))
    cax = ax.matshow(matrix, cmap='YlGnBu')
    fig.colorbar(cax)

    ax.set_xticks(range(len(seq2) + 1))
    ax.set_yticks(range(len(seq1) + 1))
    ax.set_xticklabels(["-"] + list(seq2))
    ax.set_yticklabels(["-"] + list(seq1))
    ax.set_xlabel("Sequence 2")
    ax.set_ylabel("Sequence 1")
    ax.set_title("Smith-Waterman Local Alignment Score Matrix")

    ax.plot(max_pos[1], max_pos[0], "ro")  # Highlight max score cell
    plt.show()

# Example usage
seq1 = "ACACACTA"
seq2 = "AGCACACA"
score_matrix, max_pos = smith_waterman(seq1, seq2)
plot_score_matrix(score_matrix, seq1, seq2, max_pos)


## 5. Global Sequence Alignment (Done)


## 💡 What's the Goal?
We want to line up two sequences (like `GATTACA` and `GCATGCU`) to find the best match between them — from start to end.

For example:
G A T T A C A
| | | | |
G C - A T G C U


## Scoring System
We give:
- **+1 point** for a match
- **-1 point** for a mismatch
- **-2 points** for a gap (space)

In [23]:
# STEP 1: SETUP

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Score settings
match = 1
mismatch = -1
gap = -2

In [ ]:
# Jupyter Notebook Cell 5: Needleman-Wunsch Algorithm
def needleman_wunsch(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = np.zeros((m+1, n+1), dtype=int)

    # Initialize first row and column
    for i in range(1, m+1):
        dp[i][0] = dp[i-1][0] + gap
    for j in range(1, n+1):
        dp[0][j] = dp[0][j-1] + gap

    # Fill the matrix
    for i in range(1, m+1):
        for j in range(1, n+1):
            match_score = dp[i-1][j-1] + score(seq1[i-1], seq2[j-1])
            delete = dp[i-1][j] + gap
            insert = dp[i][j-1] + gap
            dp[i][j] = max(match_score, delete, insert)

    return dp


In [ ]:
# Jupyter Notebook Cell 6: Step-by-step visualization

def show_matrix(dp, seq1, seq2):
    plt.figure(figsize=(10, 6))
    ax = sns.heatmap(dp, annot=True, fmt='d', cmap="YlGnBu", cbar=False, 
                     xticklabels=['-'] + list(seq2), 
                     yticklabels=['-'] + list(seq1))
    plt.title("Alignment Score Matrix")
    plt.xlabel("Sequence 2")
    plt.ylabel("Sequence 1")
    plt.show()


In [ ]:
# Jupyter Notebook Cell 7: Try it Out!
seq1 = "GATTACA"
seq2 = "GCATGCU"

dp_matrix = needleman_wunsch(seq1, seq2)
show_matrix(dp_matrix, seq1, seq2)


In [ ]:
# Jupyter Notebook Cell 8: Traceback to Show Aligned Sequences
def traceback(dp, seq1, seq2):
    aligned1 = ""
    aligned2 = ""
    i, j = len(seq1), len(seq2)

    while i > 0 or j > 0:
        current = dp[i][j]
        if i > 0 and j > 0 and current == dp[i-1][j-1] + score(seq1[i-1], seq2[j-1]):
            aligned1 = seq1[i-1] + aligned1
            aligned2 = seq2[j-1] + aligned2
            i -= 1
            j -= 1
        elif i > 0 and current == dp[i-1][j] + gap:
            aligned1 = seq1[i-1] + aligned1
            aligned2 = "-" + aligned2
            i -= 1
        else:
            aligned1 = "-" + aligned1
            aligned2 = seq2[j-1] + aligned2
            j -= 1

    return aligned1, aligned2


In [ ]:
# Jupyter Notebook Cell 9: Show Final Result
aligned1, aligned2 = traceback(dp_matrix, seq1, seq2)

print("🔬 Final Alignment:\n")
print(aligned1)
print(aligned2)


In [ ]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-2):
    n, m = len(seq1), len(seq2)
    score = np.zeros((n+1, m+1), dtype=int)

    # Initialization
    for i in range(n+1):
        score[i][0] = i * gap
    for j in range(m+1):
        score[0][j] = j * gap

    # Fill the scoring matrix
    for i in range(1, n+1):
        for j in range(1, m+1):
            diag = score[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            delete = score[i-1][j] + gap
            insert = score[i][j-1] + gap
            score[i][j] = max(diag, delete, insert)

    # Traceback
    aligned_seq1, aligned_seq2 = '', ''
    i, j = n, m
    while i > 0 and j > 0:
        current = score[i][j]
        if current == score[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch):
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif current == score[i-1][j] + gap:
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1

    while i > 0:
        aligned_seq1 = seq1[i-1] + aligned_seq1
        aligned_seq2 = '-' + aligned_seq2
        i -= 1
    while j > 0:
        aligned_seq1 = '-' + aligned_seq1
        aligned_seq2 = seq2[j-1] + aligned_seq2
        j -= 1

    return score, aligned_seq1, aligned_seq2

# Example
score_matrix, a1, a2 = needleman_wunsch("GATTACTA", "GCATGCU")
print("\nAlignment:")
print(a1)
print(a2)

## 6. Comparison Table

| Feature         | Needleman-Wunsch       | Smith-Waterman        |
|----------------|-------------------------|------------------------|
| Type            | Global Alignment        | Local Alignment        |
| Suitable for    | Similar-length sequences| Dissimilar regions     |
| Initialization  | Penalize gaps at start  | Start with zeros       |
| Traceback Start | Bottom-right of matrix  | Max scoring cell       |

## 7. BLAST 
Why blast is fast & 
Why blast is incomplete 